In [1]:
from google.colab import files
file = files.upload()

Saving train_data.csv to train_data.csv


In [2]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import os
import numpy as np
import pandas as pd

train_data = pd.read_csv("train_data.csv")
train_data = train_data.iloc[:, 1:]

train_data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Action,time,Event
0,0.62,0.03,0.10,0.28,0.43,0.67,0.38,0.12,0.15,0.88,0.72,0.89,0.43,0.36,0.43,0.66,0.16,0,0.47,1
1,0.12,0.87,0.43,0.28,0.02,0.23,0.82,0.14,0.81,0.68,0.53,0.24,0.05,0.01,0.96,0.98,0.34,0,1.39,1
2,0.37,0.06,0.47,0.86,0.86,0.33,0.01,0.90,0.79,0.43,0.64,0.84,0.05,0.56,0.01,0.77,0.62,0,3.61,1
3,0.57,0.39,0.65,0.76,0.83,0.92,0.57,0.60,0.34,0.35,0.44,0.93,0.44,0.65,0.61,0.25,0.96,0,1.02,1
4,0.82,0.00,0.26,0.27,0.82,0.68,0.43,0.92,0.81,0.26,0.66,0.04,0.10,0.93,0.82,0.37,0.09,1,2.07,1


In [3]:
train_data = train_data.drop(np.where(train_data.time == 0)[0].tolist())
train_data

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Action,time,Event
0,0.62,0.03,0.10,0.28,0.43,0.67,0.38,0.12,0.15,0.88,0.72,0.89,0.43,0.36,0.43,0.66,0.16,0,0.47,1
1,0.12,0.87,0.43,0.28,0.02,0.23,0.82,0.14,0.81,0.68,0.53,0.24,0.05,0.01,0.96,0.98,0.34,0,1.39,1
2,0.37,0.06,0.47,0.86,0.86,0.33,0.01,0.90,0.79,0.43,0.64,0.84,0.05,0.56,0.01,0.77,0.62,0,3.61,1
3,0.57,0.39,0.65,0.76,0.83,0.92,0.57,0.60,0.34,0.35,0.44,0.93,0.44,0.65,0.61,0.25,0.96,0,1.02,1
4,0.82,0.00,0.26,0.27,0.82,0.68,0.43,0.92,0.81,0.26,0.66,0.04,0.10,0.93,0.82,0.37,0.09,1,2.07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,0.50,0.99,0.61,0.88,0.68,0.34,0.96,1.00,0.19,0.80,0.14,0.29,0.41,0.40,0.73,0.76,0.01,1,4.77,1
662,0.62,0.19,0.51,0.30,0.12,0.10,0.41,0.25,0.17,0.17,0.73,0.20,0.18,0.10,0.19,0.39,0.17,1,0.03,1
663,0.84,0.30,0.99,0.55,0.83,0.98,0.17,0.33,0.83,0.45,0.68,0.51,0.57,0.46,0.71,0.29,0.37,0,3.60,1
664,0.94,0.83,0.37,0.69,0.92,0.62,0.39,0.41,0.20,0.74,0.60,0.46,0.21,0.06,0.23,0.93,0.26,0,6.52,1


In [5]:
# X = train_data.iloc[:, [4, 6, 10, 13, 14, 17]]
X = train_data.iloc[:, :17]
y = train_data.iloc[:, 18]

X_train, X_val, y_train, y_val = train_test_split(X, np.log(y), test_size=0.2)

X_train.shape

(525, 17)

In [22]:
input_size = X_train.shape[1]
hidden_size = [20, 50, 100, 150, ]
code_size = 32

model = Sequential()

model.add(Dense(input_size, activation='relu', input_shape=(input_size, )))

for node in hidden_size:
    model.add(Dense(node, activation='relu'))

for node in reversed(hidden_size):
    model.add(Dense(node, activation='relu'))

model.add(Dense(input_size, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mae', metrics=['mse'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 17)                306       
_________________________________________________________________
dense_63 (Dense)             (None, 20)                360       
_________________________________________________________________
dense_64 (Dense)             (None, 50)                1050      
_________________________________________________________________
dense_65 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_66 (Dense)             (None, 150)               15150     
_________________________________________________________________
dense_67 (Dense)             (None, 150)               22650     
_________________________________________________________________
dense_68 (Dense)             (None, 100)              

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp = ModelCheckpoint(filepath=checkpoint_path,
                    save_weights_only=True,
                    save_best_only=True,
                    verbose=0)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=100)

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, verbose=0, callbacks=[cp, es])  # starts training

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_loss'])